In [23]:
import pandas as pd
from sqlalchemy import create_engine
from sklearn.model_selection import train_test_split
from sklearn.metrics import auc, accuracy_score
from sklearn.ensemble import RandomForestClassifier
import datetime
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.base import BaseEstimator, TransformerMixin
import numpy as np


In [24]:
#constants
rand_state = 1337

In [41]:
def preprocess_data(df, label_cols=[], forced_cat_cols=[], drop_cols=[]):

  df = df.fillna(np.nan) #because SimpleImputer requires specification of the type of nan value, we use this generic to change all nan types to np.nan types
  #df.columns= df.columns.str.lower()

  #df[forced_cat_cols] = df[forced_cat_cols].astype(object) #change dtype of force category cols to object so get_dummies encodes it

  df = df.drop(columns=drop_cols)

  df_labels = df[label_cols] #splits any specified columns off to a label df
  df = df.drop(label_cols,axis=1)

  cat_cols = df.select_dtypes(exclude="number").columns #should include forced_cat_cols set above
  num_cols = df.select_dtypes(include="number").columns

  #impute num + cat
  for col in cat_cols:
    df[col] = SimpleImputer(strategy="most_frequent").fit_transform(df[[col]]) #used this syntax with the loop because it keeps the data as a dataframe. zero chance of column's getting mixed handling nparrays
  for col in num_cols:
    df[col] = df[col].fillna(-1)
    #df[col] = SimpleImputer(strategy="mean").fit_transform(df[[col]]) #used this syntax with the loop because it keeps the data as a dataframe. zero chance of column's getting mixed handling nparrays
  if df.isna().sum().sum() > 0:
    print(f"WARNING: {df.isna().sum().sum()} nulls still exist after imputing.")

  #scale num
  for col in num_cols:
    df[col] = StandardScaler().fit_transform(df[[col]]) #used this syntax with the loop because it keeps the data as a dataframe. zero chance of column's getting mixed handling nparrays

  df = pd.get_dummies(df)

  if len(label_cols)>0:
    return df, df_labels
  else:
    return df




def add_missing_cols(expected_cols,df):
    for col in expected_cols:
        if col not in df.columns:
            print(col, "not in df so adding - should always be categorical!")
            df[col] = 0
    return df

def sync_cols(train_df, pred_df):
    for col in train_df.columns:
      if col not in pred_df.columns:
        print(col, "not in pred_df so adding - should always be categorical!")
        pred_df[col] = 0
    for col in pred_df.columns:
      if col not in train_df.columns:
        print(col, "not in train_df so dropping")
        pred_df = pred_df.drop(pred_df[col])
    return pred_df
      

def amex_metric(y_true: pd.DataFrame, y_pred: pd.DataFrame) -> float:

    def top_four_percent_captured(y_true: pd.DataFrame, y_pred: pd.DataFrame) -> float:
        df = (pd.concat([y_true, y_pred], axis='columns')
              .sort_values('prediction', ascending=False))
        df['weight'] = df['target'].apply(lambda x: 20 if x==0 else 1)
        four_pct_cutoff = int(0.04 * df['weight'].sum())
        df['weight_cumsum'] = df['weight'].cumsum()
        df_cutoff = df.loc[df['weight_cumsum'] <= four_pct_cutoff]
        return (df_cutoff['target'] == 1).sum() / (df['target'] == 1).sum()
        
    def weighted_gini(y_true: pd.DataFrame, y_pred: pd.DataFrame) -> float:
        df = (pd.concat([y_true, y_pred], axis='columns')
              .sort_values('prediction', ascending=False))
        df['weight'] = df['target'].apply(lambda x: 20 if x==0 else 1)
        df['random'] = (df['weight'] / df['weight'].sum()).cumsum()
        total_pos = (df['target'] * df['weight']).sum()
        df['cum_pos_found'] = (df['target'] * df['weight']).cumsum()
        df['lorentz'] = df['cum_pos_found'] / total_pos
        df['gini'] = (df['lorentz'] - df['random']) * df['weight']
        return df['gini'].sum()

    def normalized_weighted_gini(y_true: pd.DataFrame, y_pred: pd.DataFrame) -> float:
        y_true_pred = y_true.rename(columns={'target': 'prediction'})
        return weighted_gini(y_true, y_pred) / weighted_gini(y_true, y_true_pred)

    g = normalized_weighted_gini(y_true, y_pred)
    d = top_four_percent_captured(y_true, y_pred)

    return 0.5 * (g + d)

In [29]:
df = pd.read_feather(r'../../amex-default-prediction/train_data.feather')
#reduce df for development !!!!! comment out line below for final model
df = df[:100000]

In [30]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 100000 entries, 0 to 99999
Columns: 191 entries, customer_ID to target
dtypes: category(12), datetime64[ns](1), float16(176), int8(1), object(1)
memory usage: 37.1+ MB


In [31]:
df['S_2'] = np.int8(df['S_2'].dt.month) #reduce S_2 date column to month only for seasonality

In [32]:
df['statement_num'] = df.groupby("customer_ID")['S_2'].rank(method='first', ascending=False).astype(int)
df_1 = df[df['statement_num'] == 1] #last
df_2 = df[df['statement_num'] == 2] #2nd last
df_3 = df[df['statement_num'] == 3] #3rd last


df_comb = df_1.merge(df_2, how='left', on='customer_ID', suffixes=('', '_2'))
df_comb = df_comb.merge(df_3, how='left', on='customer_ID', suffixes=('_1', '_3'))
df_comb = df_comb.drop(['statement_num_1','target_2', 'statement_num_2','target_3', 'statement_num_3'], axis=1)
df_comb.rename(columns={'target_1':'target'}, inplace=True)
df_comb

,customer_ID,S_2_1,P_2_1,D_39_1,B_1_1,B_2_1,R_1_1,S_3_1,D_41_1,B_3_1,...,D_136_3,D_137_3,D_138_3,D_139_3,D_140_3,D_141_3,D_142_3,D_143_3,D_144_3,D_145_3
0,0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f...,12,0.868652,0.001082,0.001930,1.007812,0.001772,0.100464,0.004627,0.006073,...,NaN,NaN,NaN,0.002398,0.008453,0.005554,NaN,0.001831,0.009613,0.007385
1,00000fd6641609c6ece5454664794f0340ad84dddce9a2...,12,0.861328,0.302246,0.006710,0.819824,0.007965,0.139160,0.000728,0.005234,...,NaN,NaN,NaN,0.009323,0.006409,0.009880,NaN,0.007610,0.001233,0.002165
2,00001b22f846c82c51f6e3958ccd81970162bae8b007e8...,12,0.872070,0.007484,0.001891,0.815918,0.006485,NaN,0.007050,0.007042,...,NaN,NaN,NaN,0.005657,0.001323,0.000512,NaN,0.004681,0.000129,0.006077
3,000041bdba6ecadd89a52d11886e8eaaec9325906c9723...,12,0.614258,0.007423,0.015823,0.961914,0.004478,0.369873,0.004799,0.009392,...,NaN,NaN,NaN,0.008018,0.004776,0.004845,NaN,0.000654,0.001793,0.006355
4,00007889e4fcd2614b6cbe7f8f3d2e5c728eca32d9eb8a...,12,0.877441,0.001762,0.007950,0.810547,0.003195,NaN,0.009697,0.004700,...,NaN,NaN,NaN,0.007145,0.005054,0.008698,NaN,0.003080,0.000633,0.005486
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8289,049d799511c2d516bf98c1f5f0a1a42b6487f3774a54a1...,12,0.527344,0.036224,0.725586,0.024582,0.251465,0.208374,0.008018,1.003906,...,NaN,NaN,NaN,1.001953,0.000210,0.945312,0.410156,1.006836,0.291992,0.092712
8290,049dc60e06ade9b0bf946472126eed3bb531227f60c7c4...,12,0.941406,0.001174,0.021790,1.000977,0.009361,0.088989,0.002811,0.001303,...,NaN,NaN,NaN,0.001455,0.008949,0.000666,NaN,0.002975,0.008888,0.003250
8291,049dcc14cb874dbcd3b473f17e7eb375e6f13ef591bf0f...,12,0.847656,0.009995,0.005932,0.813965,0.002266,0.147217,0.006927,0.022110,...,NaN,NaN,NaN,0.000714,0.004436,0.007732,NaN,0.000073,0.009018,0.009048
8292,049dd5e7b5a6895c4f8b179362d8e91f26ebf0b6c95e99...,12,0.460693,0.002773,0.566406,0.024063,0.001910,0.163940,0.009346,0.635742,...,NaN,NaN,NaN,0.004463,0.002012,0.001697,NaN,0.003872,0.008324,0.003468


In [33]:
df = df_comb #overwriting df so I can reuse code from rf_pipeline
del df_1
del df_2
del df_3
del df_comb

In [34]:
#build list of columns with 50 percent missing values
percent_null = df.isnull().sum() / len(df) 
half_missing_cols = percent_null[percent_null > 0.5].index

In [43]:
#build model
y = df['target']
x = df.drop(columns=["target"])

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=rand_state)

x_train, x_train_labels = preprocess_data(x_train, label_cols=["customer_ID"], drop_cols=half_missing_cols)

#x_train = add_missing_cols(x_train.columns, x_train)

rf_all = RandomForestClassifier(random_state=rand_state, n_jobs=-1)
print("Fitting Model")
rf_all.fit(x_train, y_train)


Fitting Model


RandomForestClassifier(n_jobs=-1, random_state=1337)

In [44]:
#x_test = add_missing_cols(x_train.columns, x_test)

x_test, x_test_labels = preprocess_data(x_test, label_cols=["customer_ID"], drop_cols=half_missing_cols)

x_test = sync_cols(x_train, x_test)

rf_all.score(x_test, y_test)

D_64_1_-1 not in pred_df so adding - should always be categorical!
D_64_3_-1 not in train_df so dropping


KeyError: '[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0

In [13]:
del df #free up memory

In [14]:
df = pd.read_feather(r'../../amex-default-prediction/test_data.feather')
#reduce df for development !!!!! comment out line below for final model
#df = df[:100000]


df['S_2'] = np.int8(df['S_2'].dt.month) #reduce S_2 date column to month only for seasonality

df['statement_num'] = df.groupby("customer_ID")['S_2'].rank(method='first', ascending=False).astype(int)
df_1 = df[df['statement_num'] == 1] #last
df_2 = df[df['statement_num'] == 2] #2nd last
df_3 = df[df['statement_num'] == 3] #3rd last


df_comb = df_1.merge(df_2, how='left', on='customer_ID', suffixes=('', '_2'))
df_comb = df_comb.merge(df_3, how='left', on='customer_ID', suffixes=('_1', '_3'))
df_comb = df_comb.drop(['statement_num_1', 'statement_num_2', 'statement_num_3'], axis=1)
df_comb.rename(columns={'target_1':'target'}, inplace=True)
df_comb

df = df_comb #overwriting df so I can reuse code from rf_pipeline

del df_1
del df_2
del df_3
del df_comb

In [ ]:
df, df_labels = preprocess_data(df, label_cols=["customer_ID"], drop_cols=half_missing_cols)

df = add_missing_cols(x_train, df)

In [19]:
rf_wide_prediction_preds = rf_all.predict(df)
rf_wide_prediction_proba = rf_all.predict_proba(df)
rf_wide_prediction_output = pd.concat([df_labels,pd.DataFrame(rf_wide_prediction_preds,columns=["pred"]),pd.DataFrame(rf_wide_prediction_proba,columns=["proba-inv","proba"])], axis=1)

rf_wide_prediction_output = rf_wide_prediction_output.drop(['pred', 'proba-inv'], axis=1)
rf_wide_prediction_output = rf_wide_prediction_output.rename(columns={"proba":"prediction"})

rf_wide_prediction_output.to_csv(r"..\..\amex-default-prediction\rf_wide_neg_one_output.csv",index=False)

c:\Users\Neo\AppData\Local\Programs\Python\Python38\lib\site-packages\sklearn\base.py:493: FutureWarning: The feature names should match those that were passed during fit. Starting version 1.2, an error will be raised.
Feature names must be in the same order as they were in fit.

  warnings.warn(message, FutureWarning)
c:\Users\Neo\AppData\Local\Programs\Python\Python38\lib\site-packages\sklearn\base.py:493: FutureWarning: The feature names should match those that were passed during fit. Starting version 1.2, an error will be raised.
Feature names must be in the same order as they were in fit.

  warnings.warn(message, FutureWarning)
